<a href="https://colab.research.google.com/github/ritsika/bank-customer-churn/blob/main/bank_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install eli5

In [ ]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
df = pd.read_csv('/content/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
X = df.iloc[:,3:-1]
encoder = OrdinalEncoder()
value = encoder.fit_transform(X['Geography'].values.reshape(-1, 1))
X['Geography'] = value
encoder = OrdinalEncoder()
value = encoder.fit_transform(X['Gender'].values.reshape(-1, 1))
X['Gender'] = value
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0.0,0.0,42,2,0.00,1,1,1,101348.88
1,608,2.0,0.0,41,1,83807.86,1,0,1,112542.58
2,502,0.0,0.0,42,8,159660.80,3,1,0,113931.57
3,699,0.0,0.0,39,1,0.00,2,0,0,93826.63
4,850,2.0,0.0,43,2,125510.82,1,1,1,79084.10


In [ ]:
y = df.iloc[:,len(df.columns)-1]
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)
len(X_train), len(X_test), len(y_train), len(y_test)

(7000, 3000, 7000, 3000)

In [ ]:
# Random Forest Classifier

In [ ]:
RF = RandomForestClassifier(n_estimators = 200, max_depth = 10, random_state = 0)
RF.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=200, random_state=0)

In [ ]:
# Create a DataFrame for testing the model
test_data = pd.DataFrame({
     'CreditScore': [600, 550, 620, 580, 700],
    'Geography': ['1.0', '1.0', '2.0', '2.0', '1.0'],
    'Gender': ['1.0', '0.0', '1.0', '1.0', '0.0'],
    'Age': [35, 40, 28, 32, 45],
    'Tenure': [3, 5, 2, 1, 4],
    'Balance': [2000, 1500, 100, 3000, 800],
    'NumOfProducts': [2, 1, 2, 1, 3],
    'HasCrCard': [1, 1, 0, 1, 0],
    'IsActiveMember': [0, 1, 0, 1, 0],
    'EstimatedSalary': [80000, 60000, 40000, 90000, 70000]
})

# Use the trained model to make predictions on the new data
predictions = RF.predict(test_data)

# Print the predicted churn status for the new data point
print("Predicted Churn Status:")
for index, prediction in enumerate(predictions, start=1):
    print('Customer', index, ':', prediction)

Predicted Churn Status:
Customer 1 : 0
Customer 2 : 0
Customer 3 : 0
Customer 4 : 0
Customer 5 : 1


In [ ]:
round(RF.score(X_test, y_test), 4)

0.8677

In [ ]:
perm = PermutationImportance(RF, random_state = 42, n_iter = 10).fit(X, y)
eli5.show_weights(perm, feature_names = X.columns.tolist())

Weight,Feature
0.0947 ± 0.0045,Age
0.0639 ± 0.0040,NumOfProducts
0.0449 ± 0.0039,IsActiveMember
0.0227 ± 0.0009,Balance
0.0172 ± 0.0017,Geography
0.0118 ± 0.0017,CreditScore
0.0111 ± 0.0018,EstimatedSalary
0.0098 ± 0.0019,Tenure
0.0085 ± 0.0014,Gender
0.0029 ± 0.0005,HasCrCard
